<a href="https://colab.research.google.com/github/ajaycode/machine-learning/blob/main/Content%20Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content summarization

In [ ]:
# !pip install -qq feedparser trafilatura feedgenerator

In [ ]:
import trafilatura
import feedparser
import requests
from bs4 import BeautifulSoup
from feedgenerator import DefaultFeed

## Getting Text from a URL

In [ ]:
# Download an example website
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')

In [ ]:
# Extract the main text (And show the start)
trafilatura.extract(downloaded, include_comments=False, include_tables=False)[:2048]

'Every March we recognize the women who have shaped history—and now, we’re taking a look forward. From driving software development in large companies to maintaining thriving open source communities, we’re spending Women’s History Month with women leaders who are making history every day in the tech community. Erin Spiceland is a Software Engineer for SpaceX. Born and raised in rural south Georgia, she is a Choctaw and Chickasaw mother of two now living in downtown Los Angeles. Erin didn’t finish college—she’s a predominantly self-taught software engineer. In her spare time, she makes handmade Native American beadwork and regalia and attends powwows.\nHow would you summarize your career (so far) in a single sentence?\nMy career has been a winding road through periods of stimulation and health as well as periods of personal misery. During it all, I’ve learned a variety of programming languages and technologies while working on a diverse array of products and services. I’m a domestic abu

In [ ]:
# Make a function to do this
def get_text(url):
    downloaded = trafilatura.fetch_url(url)
    return trafilatura.extract(downloaded, include_comments=False, include_tables=False)

## Sumarization

See https://huggingface.co/inference-api and get your token from https://huggingface.co/settings/tokens (or visit the model page at https://huggingface.co/facebook/bart-large-cnn, click 'Deploy', choose Inference API and click 'show token to get the code ready to go)

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
headers = {"Authorization": "Bearer YOUR_API_KEY"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
def summarize(text):
    if text is None: return None
    output = query({
        "inputs": text[:2048],
            "max_length":300, 
            "min_length":30, 
            "do_sample":False
    })
    return output[0]['summary_text']

In [ ]:
# Test this:
text = get_text('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
summarize(text)

'Erin Spiceland is a Software Engineer for SpaceX. Born and raised in rural south Georgia, she is a Choctaw and Chickasaw mother of two now living in downtown Los Angeles. In her spare time, she makes handmade Native American beadwork and regalia and attends powwows.'

## Getting and Summarizing the feed

In [ ]:
HN_Feed = feedparser.parse('https://hnrss.org/frontpage')

In [ ]:
len(HN_Feed.entries)

20

In [ ]:
# Run through the first 3 entries, getting the text and summarizing:
for p in HN_Feed.entries[:3]:
    summary = p['summary']
    if 'Article URL' in summary:
        soup = BeautifulSoup(summary, 'html.parser')
        url = soup.find('a').get('href')
        text = get_text(url)
        if text is not None:
            summary = summarize(text)
    print(f"Title: {p['title']}")
    print(f"Summary: {summary}\n\n")

Title: Where does ChatGPT fall on the political compass?
Summary: OpenAI recently released ChatGPT, one of the most impressive conversational artificial intelligence systems ever created. In the first five days after its debut on November 30, over 1 million people had already interacted with the system. We asked the hot new artificial intelligence system to take four popular political quizzes. Guess what we found...


Title: PRQL: a simple, powerful, pipelined SQL replacement
Summary: PRQL is an open-source programming language. It is database agnostic, compiling to many dialects of SQL. It has bindings to most major languages (and more are in progress) PRQL allows for column lineage and type inspection. It also allows for powerful autocomplete, type-checking, and helpful error messages.


Title: Herb Kelleher: Manage in Good Times So You'll Do Well in the Bad Times (2006)
Summary: Herb Kelleher: Manage in Good Times So You'll Do Well in the Bad Times. Southwest Airlines chairman and f

# Sharing with the World

I use the `feedgenerator` package to write the summaries to a new feed (with some custom extra steps for sites like YouTube and Twitter). You can load this in your favourite feed reader by pointing it at https://www.distilhn.com/feed/feed.xml. I also made a basi index.html file that uses javascript to read this feed and insert each item into the page, visible at https://www.distilhn.com. feel free to view the page source to see how that works :)